# Clustering

In [3]:
import pandas as pd
import numpy as np
from functools import reduce

import seaborn as sns
import matplotlib.pyplot as plt

from config import *
from clustering_utils import *

## Load data

In [4]:
features_raw = pd.read_parquet(FEATURES_DIR / "all_features.pq")
features_raw.shape

(192, 520)

In [5]:
# Drop participants where all features are null
features = features_raw[~(features_raw.isna().sum(axis=1) == features_raw.shape[1]-2)]
features.shape

(171, 520)

In [6]:
y = pd.DataFrame(features["y"], columns=["y"])
X = features.drop(["participant_id", "y"], axis=1)
X.shape

(171, 518)

### Normalize X

In [7]:
X_reduced = X.pipe(remove_low_variance_features).pipe(remove_multicollineraity, y)

/mnt/c/Users/idwe/Documents/Github/Analyzing-concussions-through-eyetracking-measurements/.venv/lib/python3.12/site-packages/sklearn/feature_selection/_variance_threshold.py:114: RuntimeWarning: Degrees of freedom <= 0 for slice.
  self.variances_ = np.nanvar(X, axis=0)
2025-05-09 10:17:12,937 - INFO - clustering_utils.remove_low_variance_features:48 - Kept 458 features out of 518
2025-05-09 10:17:13,253 - INFO - clustering_utils.remove_multicollineraity:106 - Removed 155 highly correlated features: {'SMOOTH_PURSUITS_fixations_per_second_max', 'EVIL_BASTARD_duration_max_sacc', 'EVIL_BASTARD_avg_pupil_size_max_fix', 'KING_DEVICK_saccades_per_second_mean', 'KING_DEVICK_duration_mean_fix', 'SMOOTH_PURSUITS_distance_to_fixpoint_mean', 'SHAPES_distance_to_fixpoint_median', 'SMOOTH_PURSUITS_distance_to_fixpoint_std', 'FIXATIONS_avg_pupil_size_min_fix', 'FIXATIONS_fixations_per_second_mean', 'FITTS_LAW_fixations_per_second_median', 'ANTI_SACCADE_total_acceleration_magnitude_median', 'REACTION